In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, StandardScaler, OneHotEncoder, Binarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, ConfusionMatrixDisplay, roc_auc_score, roc_curve

In [2]:
filepath = r"C:\Users\TomOldmeadow\Documents\Training\PowerBI\Tech_Report_SO2024\survey_results_public.csv"
df24 = pd.read_csv(filepath)
df24.head()

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,3,I am a developer by profession,45-54 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,...,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,NaN,NaN
3,4,I am learning to code,18-24 years old,"Student, full-time",NaN,Apples,NaN,Some college/university study without earning ...,"Other online resources (e.g., videos, blogs, f...",Stack Overflow;How-to videos;Interactive tutorial,...,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN,NaN
4,5,I am a developer by profession,18-24 years old,"Student, full-time",NaN,Apples,NaN,"Secondary school (e.g. American high school, G...","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Written Tutorial...,...,NaN,NaN,NaN,NaN,NaN,NaN,Too short,Easy,NaN,NaN


Potential influencing columns:
- Age: 'Age'

- Experience years: 'WorkExp', 'YearsCodePro'

- Developer role: 'DevType', 'JobSat'

- Companies’ tech factors (company size, remote vs office): 'OrgSize', 'RemoteWork', 'Industry'

- Documentation learnt from: 'TechDoc'

- AI tool usage: 'AISelect', 'AISent', 'AIBen', 'AIAcc', 'AIThreat', 'AIEthics', 'AIChallenges'

- education level: 'EdLevel'

- language preferences: 'Language', 'ToolsTech'

In [14]:
df24_ext = df24[['Age', 'WorkExp', 'YearsCodePro', 'DevType', 'JobSat', 'OrgSize',
                 'RemoteWork', 'Industry', 'TechDoc', 'AISelect', 'AISent',
                 'AIBen', 'AIAcc', 'AIThreat', 'AIEthics', 'AIChallenges',
                 'EdLevel']]

df24_ext.head()

,Age,WorkExp,YearsCodePro,DevType,JobSat,OrgSize,RemoteWork,Industry,TechDoc,AISelect,AISent,AIBen,AIAcc,AIThreat,AIEthics,AIChallenges,EdLevel
0,Under 18 years old,NaN,NaN,NaN,NaN,NaN,Remote,NaN,NaN,Yes,Very favorable,Increase productivity,NaN,NaN,NaN,NaN,Primary/elementary school
1,35-44 years old,17.0,17,"Developer, full-stack",NaN,NaN,Remote,NaN,API document(s) and/or SDK document(s);User gu...,"No, and I don't plan to",NaN,NaN,NaN,NaN,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
2,45-54 years old,NaN,27,Developer Experience,NaN,NaN,Remote,NaN,API document(s) and/or SDK document(s);User gu...,"No, and I don't plan to",NaN,NaN,NaN,NaN,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
3,18-24 years old,NaN,NaN,"Developer, full-stack",NaN,NaN,NaN,NaN,NaN,Yes,Very favorable,Increase productivity;Greater efficiency;Impro...,Somewhat trust,No,Circulating misinformation or disinformation;M...,Don’t trust the output or answers,Some college/university study without earning ...
4,18-24 years old,NaN,NaN,"Developer, full-stack",NaN,NaN,NaN,NaN,API document(s) and/or SDK document(s);User gu...,"No, and I don't plan to",NaN,NaN,NaN,NaN,NaN,NaN,"Secondary school (e.g. American high school, G..."


In [11]:
df24_ext['AIAcc'].value_counts()

AIAcc
Somewhat trust                15039
Neither trust nor distrust     9920
Somewhat distrust              8395
Highly distrust                2932
Highly trust                   1016
Name: count, dtype: int64

In [15]:
# Remove null vaues from the AIAcc i.e. trust in AI column
df24_ext = df24_ext.dropna(subset=['AIAcc'], axis=0)

# Creating the target variable 'AITrust'
df24_ext['AITrust'] = df24_ext['AIAcc'].apply(lambda x: 1 if x in ['Highly trust', 'Somewhat trust'] else 0)

In [16]:
df24_ext['AITrust'].value_counts()

AITrust
0    21247
1    16055
Name: count, dtype: int64

In [ ]:
# Check for none values across the columns by percentage
(df24_ext.isnull().mean() * 100).round(2).sort_values(ascending=False)

Industry        51.63
JobSat          51.17
WorkExp         50.34
TechDoc         34.47
OrgSize         23.40
AIChallenges    17.83
YearsCodePro    16.76
RemoteWork      15.48
AIEthics         9.69
DevType          5.05
EdLevel          3.94
AIThreat         2.72
AIBen            1.29
AISent           0.07
Age              0.00
AISelect         0.00
AIAcc            0.00
AITrust          0.00
dtype: float64

**Based off of high null values, remove:**
- Industry
- JobSat
- WorkExp (Covered similarly by YearsCodePro)
- Techdoc (Perhaps not so relevant anyway and high nulls not worth the time)
- OrgSize (Potentially useful but too sparse)

**Based off moderate null values:**
- AIEthics (String information not relevant to whether or not someone trusts AI, but rather how) -> remove
- AIChallenges (Similar problem to AIEthics) -> Remove
- RemoteWork -> Impute values - Use 'unknown' bucket to categorise none values (standard practice)
- YearsCodePro -> Impute values - Use median value to fill nulls (standard practice)

**Low nulls < 5%**
- DevType          5.05 -> Drop rows with missing values - Also consider a cutoff point for number of data points collected for each DevType later in model Development
- EdLevel          3.94 -> Drop rows with missing values
- AIThreat         2.72 -> Drop rows with missing values
- AIBen            1.29 -> Drop column - does not directly affect feeling toward AI trust but rather questions their used tools
- AISent           0.07 -> Drop rows with missing values
- Age              0.00
- AISelect         0.00
- AIAcc            0.00

In [35]:
df24_ext['DevType'].value_counts()

DevType
Developer, full-stack                            11621
Developer, back-end                               5990
Student                                           3079
Developer, front-end                              2235
Developer, mobile                                 1275
Other (please specify):                           1234
Developer, desktop or enterprise applications     1098
Engineering manager                                781
Data scientist or machine learning specialist      721
Academic researcher                                691
Developer, embedded applications or devices        690
Data engineer                                      657
DevOps specialist                                  595
Senior Executive (C-Suite, VP, etc.)               577
Research & Development role                        496
Developer, AI                                      434
Cloud infrastructure engineer                      377
Developer, game or graphics                        330
De